# HSE 2023: Введение в машинное обучение БИ 23/24

## ДЗ 2

# Внимание!
Если в задании просят объяснить что-либо, то это значит, что требуется письменный ответ, который является частью задания и оценивается

Мы только принимаем ipynb ноутбуки. Если вы используете Google Colab, то вам необходимо скачать ноутбук перед сдачей ДЗ


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import datasets
# from sklearn.datasets import load_boston
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
!pip install category_encoders
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLSResults
from math import sqrt
import random
import sys

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

sns.set(style="darkgrid")

### Данные

Для этого ДЗ мы будем использовать датасет треков со стримингового сервиса Spotify

**Описание данных**

- **track_id:** The Spotify ID for the track
- **artists:** The artists' names who performed the track. If there is more than one artist, they are separated by a ;
- **album_name:** The album name in which the track appears
- **track_name:** Name of the track
- **popularity:** The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity.
- **duration_ms:** The track length in milliseconds
- **explicit:** Whether or not the track has explicit lyrics (true = yes it does; false = no it does not OR unknown)
- **danceability:** Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable
- **key:** The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1
- **loudness:** The overall loudness of a track in decibels (dB)
- **mode:** Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0
- **speechiness:** Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks
- **acousticness:** A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic
- **instrumentalness:** Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content
- **liveness:** Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live
- **valence:** A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry)
- **tempo:** The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration
- **time_signature:** An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of 3/4, to 7/4.
- **track_genre:** The genre in which the track belongs

**Целевая переменная**
- **energy:** Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale

## Удаление некоторых признаков
Из описания признаков можно сделать вывод, что целевая переменная energy не зависит от того, кто исполнил трек, как он называется и какому альбому принадлежит. Скорее, от этого может зависеть популярность, а она у нас уже посчитана, поэтому я удалил эти признаки из датасета

In [ ]:
data = pd.read_csv('dataset.csv')
data = data.drop(['artists', 'track_name', 'album_name'], axis=1) # удаление ненужных признаков

y = data['energy']
X = data.drop(['energy'], axis=1)
columns = X.columns

In [ ]:
data.head()

## Линейная регрессия

#### 0. [0.25 балла] Закодируйте категориальные признаки. Объясните выбранный вами метод.
Так как признак explicit принимает всего 2 значения - True или False, заменим его на бинарный численный формат

In [ ]:
# ваш код здесь
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=7)

X_train['explicit'] = X_train['explicit'].astype(int)
X_test['explicit'] = X_test['explicit'].astype(int)
X_train['key'] = X_train['key'].astype(str)
X_test['key'] = X_test['key'].astype(str)

In [9]:
target_encoder = TargetEncoder()
X_train[columns] = target_encoder.fit_transform(X_train[columns], y_train)
X_test[columns] = target_encoder.transform(X_test[columns], y_test)
X_train['explicit']

X_test.head()

,popularity,duration_ms,explicit,danceability,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
94171,58,200601,0,0.664,0.676253,-11.661,1,0.0743,0.791000,0.000000,0.1160,0.4240,133.904,4,0.464480
67448,0,228133,0,0.691,0.644248,-4.770,1,0.0735,0.239000,0.000000,0.0969,0.7790,94.960,4,0.725632
14041,0,145760,0,0.543,0.636246,-3.940,1,0.1080,0.001780,0.000000,0.0407,0.6570,145.015,4,0.496083
42831,13,177880,0,0.416,0.639455,-5.188,1,0.1330,0.000044,0.000037,0.0826,0.3520,115.611,4,0.922504
42925,12,86933,0,0.345,0.676253,-8.208,0,0.4420,0.003930,0.911000,0.2140,0.0377,121.285,3,0.922504


#### 1. [0.25 балла] Разбейте данные на train и test с пропорцией 75:25 и random_state=7.

In [10]:
# ваш код здесь
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
print("I've done it before in task 0!")

I've done it before in task 0!


#### 2. [0.75 балла] Обучите модели на train'е, исключив категориальные признаки, используя библиотеку StatsModels и примените ее к test'у; используйте $RMSE$ и $R ^ 2$ в качестве метрики качества. Попробуйте также применить реализации линейной регрессии из sklearn:

* [`LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html);
* [`Ridge`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) with $\alpha = 0.03$;
* [`Lasso`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) with $\alpha = 0.05$
* [`ElasticNet`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) with $\alpha = 0.01$, $l_{1}$_$ratio = 0.4$

Не забывайте скейлить данные с помощью StandardScaler перед обучением моделей!

In [11]:
# ваш код здесь
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
categorical_features = ['explicit', 'key', 'mode', 'track_genre']
columns_copy = columns
X_train_copy = X_train
X_test_copy = X_test

X_train = X_train.drop(categorical_features, axis=1)
X_test = X_test.drop(categorical_features, axis=1)
columns = X_train.columns

scaler = StandardScaler()

X_train[columns] = scaler.fit_transform(X_train[columns])
X_test[columns] = scaler.transform(X_test[columns])

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)
model = sm.OLS(y_train, X_train).fit()
predictions_train = model.predict(X_train)
predictions_test = model.predict(X_test)

r2_train = r2_score(y_train, predictions_train)
rmse_train = np.sqrt(mean_squared_error(y_train, predictions_train))

r2_test = r2_score(y_test, predictions_test)
rmse_test = np.sqrt(mean_squared_error(y_test, predictions_test))

print("StatsModels:")
print(f"RMSE (train): {rmse_train}")
print(f"R-squared (train): {r2_train}")
print(f"RMSE (test): {rmse_test}")
print(f"R-squared (test): {r2_test}")


StatsModels:
RMSE (train): 0.12189750051952532
R-squared (train): 0.765671169091747
RMSE (test): 0.12160132585000147
R-squared (test): 0.7646449003216564


In [12]:
# LinearReg
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)
linear_reg_train_predictions = linear_reg.predict(X_train)
linear_reg_test_predictions = linear_reg.predict(X_test)

# Ridge
ridge_reg = Ridge(alpha=0.03)
ridge_reg.fit(X_train, y_train)
ridge_reg_train_predictions = ridge_reg.predict(X_train)
ridge_reg_test_predictions = ridge_reg.predict(X_test)

# Lasso
lasso_reg = Lasso(alpha=0.05)
lasso_reg.fit(X_train, y_train)
lasso_reg_train_predictions = lasso_reg.predict(X_train)
lasso_reg_test_predictions = lasso_reg.predict(X_test)

# ElasticNet
elastic_net = ElasticNet(alpha=0.01, l1_ratio=0.4)
elastic_net.fit(X_train, y_train)
elastic_net_train_predictions = elastic_net.predict(X_train)
elastic_net_test_predictions = elastic_net.predict(X_test)

In [13]:
def evaluate_model(model, y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"{model}:")
    print(f"RMSE: {rmse}")
    print(f"R-squared: {r2}")

In [14]:
evaluate_model("Linear Regression (train)", y_train, linear_reg_train_predictions)
evaluate_model("Linear Regression (test)", y_test, linear_reg_test_predictions)
print()

evaluate_model("Ridge (train)", y_train, ridge_reg_train_predictions)
evaluate_model("Ridge (test)", y_test, ridge_reg_test_predictions)
print()

evaluate_model("Lasso (train)", y_train, lasso_reg_train_predictions)
evaluate_model("Lasso (test)", y_test, lasso_reg_test_predictions)
print()

evaluate_model("ElasticNet (train)", y_train, elastic_net_train_predictions)
evaluate_model("ElasticNet (test)", y_test, elastic_net_test_predictions)

Linear Regression (train):
RMSE: 0.1218975005195253
R-squared: 0.765671169091747
Linear Regression (test):
RMSE: 0.12160132585000147
R-squared: 0.7646449003216564

Ridge (train):
RMSE: 0.12189750051954563
R-squared: 0.7656711690916689
Ridge (test):
RMSE: 0.12160132516824915
R-squared: 0.7646449029606719

Lasso (train):
RMSE: 0.14788208673493783
R-squared: 0.6551205915214755
Lasso (test):
RMSE: 0.1471071204791906
R-squared: 0.6555593645544675

ElasticNet (train):
RMSE: 0.12280012036170551
R-squared: 0.7621880305586002
ElasticNet (test):
RMSE: 0.12247697499078816
R-squared: 0.7612431198019444


## Общий вывод
Глядя на полученные значения можно сказать, что модели как-то описывают зависимость - не сказать, что хорошо, но и не плохо. Посмотрим, что будет, если добавить категоральные признаки

#### 3. [0.25 балла] Повторите шаги из предыдущего пункта, добавив категориальные признаки. Прокомментируйте изменения значений метрик качества

In [15]:
X_train = X_train_copy
X_test = X_test_copy
columns = columns_copy

X_train[columns] = scaler.fit_transform(X_train[columns])
X_test[columns] = scaler.transform(X_test[columns])

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)
model = sm.OLS(y_train, X_train).fit()
predictions_train = model.predict(X_train)
predictions_test = model.predict(X_test)

r2_train = r2_score(y_train, predictions_train)
rmse_train = np.sqrt(mean_squared_error(y_train, predictions_train))

r2_test = r2_score(y_test, predictions_test)
rmse_test = np.sqrt(mean_squared_error(y_test, predictions_test))

print("StatsModels:")
print(f"RMSE (train): {rmse_train}")
print(f"R-squared (train): {r2_train}")
print(f"RMSE (test): {rmse_test}")
print(f"R-squared (test): {r2_test}")

StatsModels:
RMSE (train): 0.11760025940577189
R-squared (train): 0.7819014973115574
RMSE (test): 0.11783606225847269
R-squared (test): 0.7789943202086159


In [16]:
# LinearReg
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)
linear_reg_train_predictions = linear_reg.predict(X_train)
linear_reg_test_predictions = linear_reg.predict(X_test)

# Ridge
ridge_reg = Ridge(alpha=0.03)
ridge_reg.fit(X_train, y_train)
ridge_reg_train_predictions = ridge_reg.predict(X_train)
ridge_reg_test_predictions = ridge_reg.predict(X_test)

# Lasso
lasso_reg = Lasso(alpha=0.05)
lasso_reg.fit(X_train, y_train)
lasso_reg_train_predictions = lasso_reg.predict(X_train)
lasso_reg_test_predictions = lasso_reg.predict(X_test)

# ElasticNet
elastic_net = ElasticNet(alpha=0.01, l1_ratio=0.4)
elastic_net.fit(X_train, y_train)
elastic_net_train_predictions = elastic_net.predict(X_train)
elastic_net_test_predictions = elastic_net.predict(X_test)

In [17]:
evaluate_model("Linear Regression (train)", y_train, linear_reg_train_predictions)
evaluate_model("Linear Regression (test)", y_test, linear_reg_test_predictions)
print()

evaluate_model("Ridge (train)", y_train, ridge_reg_train_predictions)
evaluate_model("Ridge (test)", y_test, ridge_reg_test_predictions)
print()

evaluate_model("Lasso (train)", y_train, lasso_reg_train_predictions)
evaluate_model("Lasso (test)", y_test, lasso_reg_test_predictions)
print()

evaluate_model("ElasticNet (train)", y_train, elastic_net_train_predictions)
evaluate_model("ElasticNet (test)", y_test, elastic_net_test_predictions)

Linear Regression (train):
RMSE: 0.11760025940577189
R-squared: 0.7819014973115574
Linear Regression (test):
RMSE: 0.11783606225847268
R-squared: 0.7789943202086159

Ridge (train):
RMSE: 0.11760025940579118
R-squared: 0.781901497311486
Ridge (test):
RMSE: 0.11783606206176994
R-squared: 0.7789943209464617

Lasso (train):
RMSE: 0.1437496456791906
R-squared: 0.6741260158293427
Lasso (test):
RMSE: 0.14335088843982685
R-squared: 0.6729246833823286

ElasticNet (train):
RMSE: 0.1186324423490295
R-squared: 0.7780561743643593
ElasticNet (test):
RMSE: 0.11887099201676296
R-squared: 0.7750951780597408


## Вывод
Модели стали чуть лучше и это говорит о том, что категоральные фичи оказывают воздействие на целевую переменную energy.

#### 4. [1 балл] Исследуйте значения параметров полученных моделей и проверьте какие веса получились нулевыми. Прокомментируйте значимость коэффициентов, обшую значимость модели и остальные факторы из результирующей таблицы

In [18]:
# ваш код здесь
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
model.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                   Results: Ordinary least squares
=====================================================================
Model:              OLS              Adj. R-squared:     0.782       
Dependent Variable: energy           AIC:                -123348.8613
Date:               2023-10-23 12:44 BIC:                -123199.1609
No. Observations:   85500            Log-Likelihood:     61690.      
Df Model:           15               F-statistic:        2.043e+04   
Df Residuals:       85484            Prob (F-statistic): 0.00        
R-squared:          0.782            Scale:              0.013832    
---------------------------------------------------------------------
                     Coef.  Std.Err.     t     P>|t|   [0.025  0.975]
---------------------------------------------------------------------
const                0.6420   0.0004 1596.1416 0.0000  0.6412  0.6428
popularity          -0.0022   0.0004   -5.3965 0.0000 -0.0030 -0.0014
duration_ms          0.0031   0.0004    7.4851 0.0000  0.0023  0.0039
explicit            -0.0026   0.0004   -6.0042 0.0000 -0.0034 -0.0017
danceability        -0.0321   0.0005  -65.5718 0.0000 -0.0331 -0.0312
key                  0.0034   0.0004    8.3484 0.0000  0.0026  0.0042
loudness             0.1223   0.0006  204.1852 0.0000  0.1211  0.1234
mode                -0.0031   0.0004   -7.6638 0.0000 -0.0039 -0.0023
speechiness          0.0239   0.0004   54.2151 0.0000  0.0230  0.0247
acousticness        -0.0861   0.0006 -148.9532 0.0000 -0.0872 -0.0850
instrumentalness     0.0340   0.0005   71.5499 0.0000  0.0331  0.0349
liveness             0.0239   0.0004   56.5779 0.0000  0.0231  0.0247
valence              0.0417   0.0005   85.0343 0.0000  0.0407  0.0426
tempo                0.0053   0.0004   12.5814 0.0000  0.0045  0.0061
time_signature       0.0052   0.0004   12.3937 0.0000  0.0044  0.0060
track_genre          0.0438   0.0006   78.4051 0.0000  0.0427  0.0449
---------------------------------------------------------------------
Omnibus:             5840.817       Durbin-Watson:          1.994    
Prob(Omnibus):       0.000          Jarque-Bera (JB):       26529.446
Skew:                0.167          Prob(JB):               0.000    
Kurtosis:            5.708          Condition No.:          3        
=====================================================================
Notes:
[1] Standard Errors assume that the covariance matrix of the errors
is correctly specified.
"""

Посмотрим на подобранные веса линейной регрессией и на значение свободного члена

In [74]:
print(linear_reg.coef_)
for coef in linear_reg.coef_:
  print(coef == 0)
print(linear_reg.intercept_)

[ 0.         -0.00219675  0.00309853 -0.00258008 -0.03214783  0.00341422
  0.12227501 -0.0031295   0.02387998 -0.08610606  0.03399978  0.02389735
  0.04167734  0.00529321  0.00517989  0.04379556]
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
0.6420033217438598


Коэффициенты примерно совпадают с моделью из StatsModel. Посмотрим на остальные модели

In [20]:
print(ridge_reg.coef_)
for coef in linear_reg.coef_:
  print(coef == 0)
print(ridge_reg.intercept_)

[ 0.         -0.00219675  0.00309853 -0.00258008 -0.0321478   0.00341422
  0.12227494 -0.0031295   0.02387996 -0.08610605  0.03399974  0.02389735
  0.04167732  0.00529322  0.00517989  0.04379558]
0.6420033217438598


In [75]:
print(lasso_reg.coef_)
for coef in linear_reg.coef_:
  print(coef == 0)
print(lasso_reg.intercept_)

[ 0.         -0.          0.          0.         -0.          0.
  0.0867412  -0.          0.         -0.06455534  0.          0.
  0.          0.          0.          0.03001922]
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
0.6420033217438598


In [76]:
print(elastic_net.coef_)
for coef in linear_reg.coef_:
  print(coef == 0)
print(elastic_net.intercept_)

[ 0.         -0.          0.         -0.         -0.02248145  0.0001732
  0.11579863 -0.          0.0186173  -0.08532101  0.02585687  0.02190828
  0.03251377  0.00405218  0.00105692  0.04430352]
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
0.6420033217438598


## Общий вывод
Глядя на коэффициенты признаков, я могу сделать следующие выводы:

* Наиболее значимые параметры: **danceability, loudness, acousticness, instrumentalness, valence, track_genre** (для себя я выбрал критерий |t| > 65). Ну и константа, разумеется
* Наименее значимые параметры: **popularity, explicit, duration_ms, key, mode**  (|t| < 10)
* Категоральные признаки не сильно влияют на качество модели - значение коэффициента детерминации после их добавление лишь немного увеличилось

**В целом, модель можно считать значимой.**

#### 5. [1 балл] Реализуйте один из алгоритмов отбора признаков (Elimination by P-value, Forward elimination, Backward elimination), сделайте выводы.

Был выбран метод Backward elimination

In [23]:
# your code here
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
min_pvalue = 0.05 # пороговое значение p-value
while True:
  max_pvalue = model.pvalues.max()
  if max_pvalue > min_pvalue:
    print(max_pvalue)
    feature_to_remove = model.pvalues.idxmax()
    print(feature_to_remove)
    X_train = X_train.drop(feature_to_remove, axis=1)
    model = sm.OLS(y_train, X_train).fit()
  else:
    break

In [24]:
model.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                   Results: Ordinary least squares
=====================================================================
Model:              OLS              Adj. R-squared:     0.782       
Dependent Variable: energy           AIC:                -123348.8613
Date:               2023-10-23 12:44 BIC:                -123199.1609
No. Observations:   85500            Log-Likelihood:     61690.      
Df Model:           15               F-statistic:        2.043e+04   
Df Residuals:       85484            Prob (F-statistic): 0.00        
R-squared:          0.782            Scale:              0.013832    
---------------------------------------------------------------------
                     Coef.  Std.Err.     t     P>|t|   [0.025  0.975]
---------------------------------------------------------------------
const                0.6420   0.0004 1596.1416 0.0000  0.6412  0.6428
popularity          -0.0022   0.0004   -5.3965 0.0000 -0.0030 -0.0014
duration_ms          0.0031   0.0004    7.4851 0.0000  0.0023  0.0039
explicit            -0.0026   0.0004   -6.0042 0.0000 -0.0034 -0.0017
danceability        -0.0321   0.0005  -65.5718 0.0000 -0.0331 -0.0312
key                  0.0034   0.0004    8.3484 0.0000  0.0026  0.0042
loudness             0.1223   0.0006  204.1852 0.0000  0.1211  0.1234
mode                -0.0031   0.0004   -7.6638 0.0000 -0.0039 -0.0023
speechiness          0.0239   0.0004   54.2151 0.0000  0.0230  0.0247
acousticness        -0.0861   0.0006 -148.9532 0.0000 -0.0872 -0.0850
instrumentalness     0.0340   0.0005   71.5499 0.0000  0.0331  0.0349
liveness             0.0239   0.0004   56.5779 0.0000  0.0231  0.0247
valence              0.0417   0.0005   85.0343 0.0000  0.0407  0.0426
tempo                0.0053   0.0004   12.5814 0.0000  0.0045  0.0061
time_signature       0.0052   0.0004   12.3937 0.0000  0.0044  0.0060
track_genre          0.0438   0.0006   78.4051 0.0000  0.0427  0.0449
---------------------------------------------------------------------
Omnibus:             5840.817       Durbin-Watson:          1.994    
Prob(Omnibus):       0.000          Jarque-Bera (JB):       26529.446
Skew:                0.167          Prob(JB):               0.000    
Kurtosis:            5.708          Condition No.:          3        
=====================================================================
Notes:
[1] Standard Errors assume that the covariance matrix of the errors
is correctly specified.
"""

На выбранном уровне значимости 0.05 все признаки значимые, удалений не произошло, модель не изменилась

#### 6. [1 балл] Найдите лучший (по RMSE) $\alpha$ для регрессиии Lasso, используя кросс-валидацию на 5 фолдов. Вы должны выбрать значение из промежутка $[10^{-4}, 10^{3}]$.

In [25]:
# your code here
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
alphas = np.logspace(-4, 3, num=1000)
lasso = Lasso()
param_grid = {'alpha': alphas}

grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)

best_alpha = grid_search.best_estimator_
best_rmse = np.sqrt(-grid_search.best_score_)

print(f"Лучший alpha: {best_alpha}")
print(f"Лучшее RMSE: {best_rmse}")

Лучший alpha: Lasso(alpha=0.0001)
Лучшее RMSE: 0.11763473348233339


## Градиентный спуск

#### 7. [3.5 балла] Имплементируйте  Ridge регрессию для MSE loss, обученную на градиентом спуске.


Все вычисления должны быть векторизованы, а циклы Python можно использовать только для итераций градиентного спуска. В качестве критерия остановки необходимо использовать (одновременно):

* проверка абсолютной нормы разницы весов на двух соседних итерациях (например, меньше некоторого малого числа порядка $10^{-6}$, заданного параметром `tolerance`);

* достижение максимального количества итераций (например, 10000, заданного параметром `max_iter`).

Вам необходимо выполнить:

* Полный градиентный спуск:

$$
w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} Q(w_{k}).
$$

* Стохастический градиентный спуск:

$$
w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} q_{i_{k}}(w_{k}).
$$

$\nabla_{w} q_{i_{k}}(w_{k}) \, $ является оценкой градиента по набору объектов, выбранных случайным образом.

* Momentum method:

$$
h_0 = 0, \\
h_{k + 1} = \alpha h_{k} + \eta_k \nabla_{w} Q(w_{k}), \\
w_{k + 1} = w_{k} - h_{k + 1}.
$$

* Adagrad method:

$$
G_0 = 0, \\
G_{k + 1} = G_{k} + (\nabla_{w} Q(w_{k+1}))^2, \\
w_{k + 1} = w_{k} - \eta * \frac{\nabla_{w} Q(w_{k+1})}{\sqrt{G_{k+1} + \epsilon}}.
$$

Чтобы убедиться, что процесс оптимизации действительно выполняется, мы будем использовать атрибут класса `loss_history`. После вызова метода fit он должен содержать значения функции потерь для всех итераций, начиная с первой (до первого шага по антиградиенту).


Вам нужно инициализировать веса случайным вектором из нормального распределения. Ниже приведен шаблон, который должен содержать код, реализующий все варианты моделей.

In [71]:
from sklearn.base import BaseEstimator

# часть кода была написана с помощью chatGPT
class LinReg(BaseEstimator):
    def __init__(self, delta=1.0, gd_type='Momentum',
                 tolerance=1e-4, max_iter=1000, w0=None, eta=1e-2, alpha=1e-3, reg_cf=1.0, epsilon=1e-8):
        """
        gd_type: str
            'GradientDescent', 'StochasticDescent', 'Momentum', 'Adagrad'
        delta: float
            proportion of object in a batch (for stochastic GD)
        tolerance: float
            for stopping gradient descent
        max_iter: int
            maximum number of steps in gradient descent
        w0: np.array of shape (d)
            init weights
        eta: float
            learning rate
        alpha: float
            momentum coefficient
        reg_cf: float
            regularization coefficient
        epsilon: float
            numerical stability
        """

        self.delta = delta
        self.gd_type = gd_type
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.eta = eta
        self.loss_history = None # list of loss function values at each training iteration
        self.reg_cf = reg_cf
        self.epsilon = epsilon

        if self.gd_type == 'Momentum':
            self.momentum = np.zeros(X.shape[1] + 1)

    def fit(self, X, y):
        """
        X: np.array of shape (l, d)
        y: np.array of shape (l)
        ---
        output: self
        """
        if self.w0 is None:
            self.w0 = np.random.randn(X.shape[1])
        self.w = self.w0.copy()
        self.loss_history = []

        for i in range(self.max_iter):
          gradient = self.calc_gradient(X, y)
          self.w -= self.eta * gradient

          self.w -= 2 * self.eta * self.reg_cf * self.w

          loss = self.calc_loss(X, y)
          self.loss_history.append(loss)

          if i > 0 and np.linalg.norm(self.w - self.w0) < self.tolerance:
            break
          self.w0 = self.w

        return self

    def predict(self, X):
        if self.w is None:
            raise Exception('Not trained yet')
        return np.dot(X, self.w)

    def calc_gradient(self, X, y):
        """
        X: np.array of shape (l, d) (l can be equal to 1 if stochastic)
        y: np.array of shape (l)
        ---
        output: np.array of shape (d)
        """
        rows = X.shape[0]

        if self.gd_type == 'GradientDescent':
          gradient = (2.0 / rows) * np.dot(X.T, np.dot(X, self.w) - y)
        elif self.gd_type == 'StochasticDescent':
          indx = np.random.choice(rows, int(self.delta * rows), replace=False)
          X_batch = X[indx]
          y_batch = y[indx]
          gradient = (2.0 / len(X_batch)) * np.dot(X_batch.T, np.dot(X_batch, self.w) - y_batch)
        elif self.gd_type == 'Momentum':
          self.momentum = self.alpha * self.momentum + self.eta * (2.0 / rows) * np.dot(X.T, np.dot(X, self.w) - y)
          gradient = self.momentum
        else:
          if not hasattr(self, 'G'):
            self.G = np.zeros(X.shape[1])
          gradient = (2.0 / rows) * np.dot(X.T, np.dot(X, self.w) - y)
          self.G += gradient ** 2
          adj_eta = self.eta / (np.sqrt(self.G) + self.epsilon)
          gradient = adj_eta * gradient

        return gradient

    def calc_loss(self, X, y):
        """
        X: np.array of shape (l, d)
        y: np.array of shape (l)
        ---
        output: float
        """
        rows = X.shape[0]
        y_pred = np.dot(X, self.w)
        loss = (1.0 / rows) * np.sum((y_pred - y) ** 2) + self.reg_cf * np.sum(self.w ** 2)
        return loss

#### 8. [1 балл] Натренируйте и провалидируйте "ручные" модели на тех же даннных, сравните качество с моделями из Sklearn и StatsModels. Исследуйте влияние параметров `max_iter` и `alpha` на процесс оптимизации. Соответствует ли оно вашим ожиданиям?

In [73]:
# your code here
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

#### 9. [1 балл] Постройте графики (там же) зависимости значения функции потерь от номера итерации для всех моделей (полного градиентого спуска, стохастического гс, Momentum и Adagrad). Сделайте выводы о скорости сходимости различных модификаций градиентного спуска.


Не забывайте о том, как должен выглядеть *красивый* график!

In [ ]:
# your code here
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ